<a href="https://colab.research.google.com/github/erobrien6/OBrien_DSPN_S23/blob/main/Exercise15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [47]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

set.seed(12345)
simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      age <- runif(sample_size, age_lo, age_hi)
      x_error <- rnorm(sample_size, mean=0, sd=sd_x)
      c_error <- rnorm(sample_size, mean=0, sd=sd_c)
      x <- beta_xa*age + beta_x0 + x_error
      c <- beta_ca*age + beta_cx*x + beta_c0 + c_error   
      return(data.frame(age=age,x=x,c=c))
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,166.50847,51.23493,159.2687
2,185.09278,184.93102,759.4811
3,171.31788,83.33845,413.6686
4,186.33495,105.75062,407.7675
5,134.77772,28.84003,195.1092
6,99.96461,58.87999,245.2812


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [ ]:
install.packages('mediation')
library(mediation)

In [54]:
# Function
run_analysis <- function(dat) {
  fitM <- lm(x ~ age, data=dat)
  fitY <- lm(c ~ x + age, data=dat)
  fitMed <- mediate(fitM, fitY, treat='age', mediator='x')
  vec <- c(fitMed$d0, round(fitMed$d0.p, digits=4), fitMed$z0, round(fitMed$z0.p, digits=4))
}
vec_output <- run_analysis(dat)
vec_output


[1] 1.1318714 0.0180000 0.7272991 0.0020000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [55]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(dat)
      }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2]<=alpha) # p value for ACME
    ADE_cov =  mean(simouts[,4]<=alpha) # p value for ADE - taking mean of T/F vector, indicates how many were signif

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))

        }

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [56]:
# WRITE YOUR CODE HERE
coverage <- repeat_analysis(10, 0.01, 100, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
coverage

$ACME_cov
[1] 1

$ADE_cov
[1] 0.4

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

Print your results.

In [57]:
# 50
coverage_50 <- repeat_analysis(10, 0.01, 50, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

# 75
coverage_75 <- repeat_analysis(10, 0.01, 75, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

# 100
coverage_100 <- repeat_analysis(10, 0.01, 100, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

# 125
coverage_125 <- repeat_analysis(10, 0.01, 125, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

# 150
coverage_150 <- repeat_analysis(10, 0.01, 150, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


In [58]:
print(cbind(coverage_50, coverage_75, coverage_100, coverage_125, coverage_150))

         coverage_50 coverage_75 coverage_100 coverage_125 coverage_150
ACME_cov 0.5         1           0.9          0.7          1           
ADE_cov  0.3         0.6         0.8          0.8          1           


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> *Based on my results, I would conclude that 50-100% of the mediation analyses yielded significant results for mediated effects, and 30-100% of the mediation analyses yielded significant results for direct effects. The coverage results varied a lot based on the sample size and the simulated data, so running more simulations should give more consistent results. Overall, it seems that this power analysis was more likely to detect significant effects for the mediated rather than direct effects.* 
> 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *The direct effect is detected by examining how age predicts reading comprehension when reading experience (mediator) is held constant. When we generated the data and executed our mediation model, reading comprehension, c, had two predictor terms considering effects of age and reading experience (x), and then it had an error term. However, when we generated our equation for x, it only accounted for age and an error term.  This could mean that because c is represented by two predictors, the variance it shares with either is reduced. Therefore, the mediated effect may be significant in more simulated cases (higher probability of being significant). Moreover, the way we simulated the data allowed the mediator and outcome variable to be highly correlated, so the direct effect of age on reading comprehension may also be harder to detect because it doesn’t account for reading experience, whereas the indirect effects do.* 

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*